# Reinforcement Learning: Theory and Algorithms
* 第3章における各定理，補題，系などをまとめます．証明は初見でもこれを参考にすれば導出できるように丁寧に記します．
* 必要に応じて実装を行います．

この章では，関数近似を使って動的計画法(DP)を解く方法を見ていきます．DPは最適性を持つので，解けたら嬉しいですよね．第3章では線形関数で上手くいく特別な場合を見ましたが，ここではもっと一般的な関数近似と，DPのVIとPIを組み合わせる方法を探します．

本章では，以下のような手法を扱います．
**Fitted Q-Iteration (FQI):** 
* 値反復っぽい計算をします．$Q$関数を関数クラス $\mathcal{F}$ から回帰で見つける感じです．これは事前に集めたデータだけでできるのでオフラインRLの一つです．
**Fitted Policy Iteration (FPI):** 
* 方策反復っぽいアプローチです．これは実際に試行錯誤してデータを集める（エピソード実行）必要があります．


**この章での目標:**
*   関数近似を使ったときに，平均的にどれくらいいい感じになるかの保証 (誤差評価) をゲットしたいわけです．
	*   そのために集中係数 (Concentration Coefficient)という，ちょっと強めの仮定を使います．これは，データがいろんな状況をちゃんとカバーしてるかの度合いを示すものです．

## FQIとオフラインRL

### 準備
*   事前データ： $\mathcal{D} = \{(s_i, a_i, r_i, s'_i)\}_{i=1}^n$ 
	*   分布 $\nu$ に従って $(s_i, a_i)$ が選ばれます．そこに$r_i$ ，$s'_i$ を集めたタプルの集合です．
	*   このデータだけを使って，最適方策$\hat{\pi}$ (つまり $V(\hat{\pi})$ が $V^\star$ に近いポリシー) に近いものを求めたいわけです．
*   関数クラス $\mathcal{F}$: 値域は $[0, V_{\max}]$ です．
*   $\|f\|_{2, \nu}^2 = \mathbb{E}_{s, a \sim \nu} [f(s, a)^2]$ ：ノルム

**仮定 4.1 (集中性):** 
データ収集に使われた $\nu$ が，どんな$\pi$で訪れる可能性のある $(s, a)$ も，ある程度ちゃんとカバーしてるという仮定です．
* $\pi$ で $(s, a)$ を訪れる占有測度 $d^\pi(s, a)$ と $\nu(s, a)$ の比が，定数 $C$ で抑えられてる ($d^\pi(s, a) / \nu(s, a) \le C$) ということです．

**仮定 4.2 (固有のベルマン誤差):** 
  $\mathcal{T}f$ で関数を更新しても，$f \in \mathcal{F}$のどれかと$\epsilon_{\text{approx}, \nu}$ の誤差でしかないという仮定です．
$$
\epsilon_{\text {approx }, \nu} := \max _{f \in \mathcal{F}} \min _{f^{\prime} \in \mathcal{F}}\left\|f^{\prime}-\mathcal{T} f\right\|_{2, \nu}^2 
$$
## FQI アルゴリズム
アルゴリズムは以下のようになっています．

1.  最初に $f_0 \in \mathcal{F}$ (例えば全部ゼロとか) から始めます
2.  $t=1, 2, \dots, K$ っで，以下の計算を繰り返します
    *    $f_{t-1}$ を使って，$y_i = r_i + \gamma \max_{a'} f_{t-1}(s'_i, a')$ をデータ $\mathcal{D}$ の各サンプル $(s_i, a_i, r_i, s'_i)$ について計算します
    *    $\mathcal{F}$ の中で， $(s_i, a_i)$ に対して $f(s_i, a_i)$ が， $y_i$ に一番近くなるような関数 $f$ を以下のような式を用いて**回帰**で見つけます        
$$ 
f_t \in \operatorname{argmin}_{f \in \mathcal{F}} \sum_{i=1}^n ( f(s_i, a_i) - y_i )^2 
$$
つまり，
$$
\quad f_t \in \operatorname{argmin}_{f \in \mathcal{F}} \sum_{i=1}^n \left( f(s_i, a_i) - (r_i + \gamma \max_{a'} f_{t-1}(s'_i, a')) \right)^2 
$$
1. 最後に得られた $f_K$ を使って，$\pi^K(s) = \operatorname{argmax}_a f_K(s, a)$ を出力します

**直感?**
もし $\mathcal{F}$ が完璧で，データもほぼ無限にあれば， $f_t$ は最適ベルマン作用素の適用 $\mathcal{T} f_{t-1}$と同じになるわけです．つまり作用素の適用を無限大に繰り返していくと，$Q^\star$ を求められそうです．


## FQIの性能保証

### FQIのサンプル複雑度(定理 4.3 )
FQI を $K$ 回反復して得られた $\pi^K$ は，$1-\delta$ 以上で，以下の不等式を満たします．

$$ V^{\star}-V^{\pi^K} \leq \underbrace{\frac{1}{(1-\gamma)^2} \left( \sqrt{\frac{C \cdot (\text{なんか色々})}{n}} + \sqrt{C \cdot \epsilon_{\text{approx}, \nu}} \right)}_{\text{データ数と関数近似能力による誤差}} + \underbrace{\frac{\gamma^K V_{\max}}{1-\gamma}}_{\text{反復回数による誤差}} $$
* いい感じの関数クラス($\epsilon_{\text{approx}), \nu}$小さい，集中性小さい(C$ 小)，データ多い ($n$ 大) ，反復数多い($K$ 大)と上界は下がります．

### 定理4.3の証明

補題4.4と補題4.5を用いて証明を行います．
#### 補題4.4(誤差伝搬の制御)

もし、FQI の各反復ステップ $t$ において、生成された $f_{t+1}$ と $\mathcal{T} f_t$ との間の $L_{2, \nu}$ ノルムの誤差が $\left\|f_{t+1}-\mathcal{T} f_t\right\|_{2, \nu} \leq \varepsilon$ で抑えられていると仮定します．このとき、以下のような不等式が成立します．
$$
V^{\star} - V^{\pi^k} \leq \frac{\sqrt{C} \varepsilon}{1-\gamma} + \frac{\gamma^k V_{\max }}{(1-\gamma)^2}
$$
#### 補題4.4の証明
Performance difference lemmaを用います．








